In [13]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

from sklearn import model_selection, naive_bayes,svm
from sklearn.metrics import accuracy_score

reviews_df=pd.read_csv('Amazon_Reviews.csv')
reviews_df

,Review,Label
0,Stuning even for the non-gamer: This sound tr...,1
1,The best soundtrack ever to anything.: I'm re...,1
2,Amazing!: This soundtrack is my favorite musi...,1
3,Excellent Soundtrack: I truly like this sound...,1
4,"Remember, Pull Your Jaw Off The Floor After H...",1
...,...,...
194,A Book That Is Worth a Second Look: This book...,1
195,Best game ever: This games makes even amazing...,1
196,Guitar in Absentia: With all due respect to a...,0
197,Stiff and Smells like drying paint: You get w...,0


In [14]:
y=reviews_df['Label']
reviews_df.drop(columns='Label',inplace=True)

X_train,X_test,y_train,y_test=train_test_split(reviews_df,y,test_size=0.2,random_state=42)

X_train['Number']=range(1,160)

<ipython-input-14-e1567c18ea49>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Number']=range(1,160)


In [15]:
X_train.shape

(159, 2)

In [16]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer

tokenizer =RegexpTokenizer(r'\w+')
lemmatizer=WordNetLemmatizer()      
stemmer=PorterStemmer()

def preprocessing(review):
    
    final_tokens=' '
    
    tokens=tokenizer.tokenize(review)
    
    pure_tokens=[token.lower() for token in tokens if token.lower() not in stopwords.words('english')]
    
    stemmed_tokens=[stemmer.stem(pure_token) for pure_token in pure_tokens]
    
    final_tokens=final_tokens.join(stemmed_tokens)
    
    return final_tokens
    
X_train['Cleaned_text']=X_train['Review'].apply(preprocessing)
X_train['Cleaned_text']

<ipython-input-16-03b7d2b289ea>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Cleaned_text']=X_train['Review'].apply(preprocessing)


183    hand track hear far complet though miss memor ...
38     work mac clearli say line work mac os system d...
24     like album thought would heard song two though...
142    pattern detail sketch although excit purchas b...
141    contemporari fairytal sure delight book take c...
                             ...                        
106    authent first encount yoruba say cd realli hel...
14     aw beyond belief feel write keep other wast mo...
92     omg soulwax own wow like amaz album ever everi...
179    yet anoth unsubstanti case believ discrimin ce...
102    ye got book expect much man wrong love book ma...
Name: Cleaned_text, Length: 159, dtype: object

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=TfidfVectorizer(stop_words='english',use_idf=True)

vectorizer.fit(X_train['Cleaned_text'])

X_test['Cleaned_text']=X_test['Review'].apply(preprocessing)

X_train_TfIdf= vectorizer.transform(X_train['Cleaned_text'])

X_test_tfIdf=vectorizer.transform(X_test['Cleaned_text'])

<ipython-input-17-6283c0f1445b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['Cleaned_text']=X_test['Review'].apply(preprocessing)


In [18]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.metrics import confusion_matrix,roc_curve,roc_auc_score
import matplotlib.pyplot as plt

clf= MultinomialNB().fit(X_train_TfIdf.toarray(),y_train)

X_test['Cleaned_text']=X_test['Review'].apply(preprocessing)

y_pred=clf.predict(X_test_tfIdf.toarray())

confusion_matrix(y_test,y_pred)




<ipython-input-18-3fb464a61c57>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['Cleaned_text']=X_test['Review'].apply(preprocessing)


array([[ 8,  7],
       [ 1, 24]], dtype=int64)

In [19]:
y_proba_pred=clf.predict_proba(X_test_tfIdf.toarray())[::,1]

fpr,tpr,thresholds=roc_curve(y_test,y_proba_pred)

y_pred

array([1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0], dtype=int64)